# Combined data
## Xiaonan Wang
## 17Nov2022

In [ ]:
#%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
from os import listdir
from os.path import isfile, join
import re
import anndata
import seaborn as sns

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list(name='gene_cmap', colors=['lightgrey', 'thistle', 'red', 'darkred']) 

sc.settings.set_figure_params(dpi=80, color_map='viridis', vector_friendly=False,  dpi_save=300)

In [ ]:
# GSE138181 - Koth 2020 Development
Koth = sc.read('/nfs01data1/Groups/Wangxiaonan/share/rev_MI/GSE138181/write/GSE138181_processed.h5ad', cache=True)
# GSE153170 - Bakker 2021 Development
Bakker = sc.read('/nfs01data1/Groups/Wangxiaonan/share/rev_MI/GSE153170/write/GSE153170_processed.h5ad', cache=True)
# GSE172511 - Sun 2022 Circulation
Sun = sc.read('/nfs01data1/Groups/Wangxiaonan/share/rev_MI/GSE172511/write/GSE172511_processed.h5ad', cache=True)
# GSE159032 - Hu 2022 Nature genetics
Hu = sc.read('/nfs01data1/Groups/Wangxiaonan/share/rev_MI/GSE159032_GSE158919/write/GSE159032_GSE158919_processed_anno.h5ad', cache=True)
# GSE188511 - Kapuria 2022 Development
Kapuria = sc.read('/nfs01data1/Groups/Wangxiaonan/share/rev_MI/GSE188511/write/GSE188511_processed.h5ad', cache=True)
# GSE145980 - Ma 2021 EMBO Reports
Ma = sc.read('/nfs01data1/Groups/Wangxiaonan/share/rev_MI/GSE145980/write/GSE145980_processed.h5ad', cache=True)

In [ ]:
# adata: .X:row: cell, col: gene -> scaled matrix; .var: high variable genes; .obs: cell annotation
# adata.raw: .X: log norm, .var: all expressed genes
Koth = anndata.AnnData(X=Koth.raw.X, obs=Koth.obs, var=Koth.raw.var, obsm=Koth.obsm)
Bakker = anndata.AnnData(X=Bakker.raw.X, obs=Bakker.obs, var=Bakker.raw.var, obsm=Bakker.obsm)
Sun = anndata.AnnData(X=Sun.raw.X, obs=Sun.obs, var=Sun.raw.var, obsm=Sun.obsm)
Hu = anndata.AnnData(X=Hu.raw.X, obs=Hu.obs, var=Hu.raw.var, obsm=Hu.obsm)
Kapuria = anndata.AnnData(X=Kapuria.raw.X, obs=Kapuria.obs, var=Kapuria.raw.var, obsm=Kapuria.obsm)
Ma = anndata.AnnData(X=Ma.raw.X, obs=Ma.obs, var=Ma.raw.var, obsm=Ma.obsm)

In [ ]:
print('Koth'+str(Koth.shape))
print('Bakker'+str(Bakker.shape))
print('Sun'+str(Sun.shape))
print('Hu'+str(Hu.shape))
print('Kapuria'+str(Kapuria.shape))
print('Ma'+str(Ma.shape))

In [ ]:
adata = [Koth, Bakker, Sun, Hu, Kapuria, Ma]

In [ ]:
for i in range(len(adata)):
    print(adata[i].shape)
    adata[i].var_names_make_unique()

In [ ]:
#combine and save the raw counts
adata1 = anndata.AnnData.concatenate(*adata)

In [ ]:
adata1.shape

In [ ]:
adata1.obs.to_csv('adata_all.csv')

In [ ]:
adata1.obs['Day'] = adata1.obs['Day'].astype(str)

In [ ]:
del adata1.obs['predicted_db']

In [ ]:
adata1.write('./write/combined_inner.h5ad')

In [ ]:
adata = sc.read('./write/combined_inner.h5ad')

In [ ]:
sc.external.pp.harmony_integrate(adata, key='Index', adjusted_basis = 'X_pca_harmony_Index')

In [ ]:
sc.pp.neighbors(adata, use_rep = 'X_pca_harmony_Index')

In [ ]:
sc.tl.umap(adata)

In [ ]:
adata.write('./write/combined_inner.h5ad')

# Data Integration

In [ ]:
adata = sc.read('./write/combined_inner.h5ad')

In [ ]:
adata.raw = adata

In [ ]:
# find highly variable genes
# parameters are mainly selected depends on user preference
sc.pp.highly_variable_genes(
    adata, min_mean=0.02, max_mean=3, min_disp=0.3, batch_key='Index', inplace=True)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
print(np.sum(adata.var.highly_variable))

In [ ]:
# scale for pca
sc.pp.scale(adata)
# pca
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
sc.external.pp.harmony_integrate(adata, key='Index', adjusted_basis = 'X_pca_harmony_Index_combined')

In [ ]:
sc.pp.neighbors(adata, use_rep = 'X_pca_harmony_Index_combined')

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color= ['Condition', 'Day', 'Age', 'StudyID'], wspace=0.3)

In [ ]:
adata.write('./write/combined_inner_overall.h5ad')

In [ ]:
pd.crosstab(adata.obs['Day'], adata.obs['StudyID'])

In [ ]:
# get the cell annotation done first
sc.pl.umap(adata, color="Cell_type")

In [ ]:
sc.pl.umap(adata, color="log1p_n_genes_by_counts", color_map=cmap)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sc.pl.umap(adata, color="Cell_type", legend_loc="on data", legend_fontsize="xx-small", ax=ax, show=False)

In [ ]:
adata = sc.read('./write/combined_inner_overall.h5ad')

# Leiden clustering (v1)

In [ ]:
sc.tl.leiden(adata, resolution=0.8, key_added='Leiden_v1')

In [ ]:
sc.pl.umap(adata, color='Leiden_v1', legend_loc='on data')

In [ ]:
adata.write('./write/combined_inner_overall.h5ad')

# Filter out low n_genes cells

In [ ]:
adata = sc.read('./write/combined_inner_overall.h5ad')

In [ ]:
adata.raw = adata

In [ ]:
adata.shape

In [ ]:
adata = adata[adata.obs['Leiden_v1'] != '9'].copy()

In [ ]:
adata.shape

In [ ]:
sc.pp.neighbors(adata, use_rep = 'X_pca_harmony_Index_combined')

In [ ]:
sc.tl.umap(adata)

# Leiden clustering (v2)

In [ ]:
sc.tl.leiden(adata, resolution=2, key_added='Leiden_v2')

In [ ]:
sc.pl.umap(adata, color='Leiden_v2', legend_loc='on data')

In [ ]:
adata = sc.read('./write/combined_inner_overall_filtered.h5ad')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
sc.pl.umap(adata, color="Cell_type", legend_loc="on data", legend_fontsize="xx-small", ax=ax, show=False)
plt.savefig('Umap_zebrafish_integrated.png', bbox_inches='tight')

In [ ]:
adata.write('./write/combined_inner_overall_filtered.h5ad')

In [ ]:
adata.obs['StudyID'].value_counts()

In [ ]:
pd.crosstab(adata.obs['StudyID'], adata.obs['Cell_type'])

In [ ]:
idx1 = np.in1d(adata.obs['StudyID'], ['GSE159032', 'GSE158919'])

In [ ]:
sum(idx1)

In [ ]:
163955+18231

In [ ]:
idx2 = np.in1d(adata.obs['StudyID'], ['GSE145980', 'GSE138181', 'GSE172511', 'GSE188511', 'GSE153170'])

In [ ]:
sum(idx2)

# Cell type definition (Euclidean distance)

In [ ]:
adata1 = adata[idx1,:].copy()
adata2 = adata[idx2,:].copy()

In [ ]:
X_pca1 = adata1.obsm['X_pca_harmony_Index_combined']
X_pca2 = adata2.obsm['X_pca_harmony_Index_combined']

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
D_sub = euclidean_distances(X_pca2, X_pca1)

In [ ]:
print(X_pca1.shape)
print(X_pca2.shape)
print(D_sub.shape)

In [ ]:
from collections import defaultdict
cl_assigned = []
Rstore = defaultdict(list) # dictionary to store results
for i in range(D_sub.shape[0]):
    CellDis = D_sub[i,:]
    CellDis_sorted = np.argsort(CellDis)[:15]
    cl_assigned.append(np.in1d(range(len(CellDis)), CellDis_sorted))
    Rstore['MinDist'].append(np.min(CellDis[CellDis_sorted]))
    Rstore['MedianDist'].append(np.median(CellDis[CellDis_sorted]))
    Rstore['MaxDist'].append(np.max(CellDis[CellDis_sorted]))
    Rstore['SD'].append(np.std(CellDis[CellDis_sorted]))
    Rstore['CT'].append(adata1[CellDis_sorted,:].obs['Cell_type'].value_counts().index[0])
Rstore = pd.DataFrame.from_dict(Rstore)
Rstore.index = adata2.obs_names

In [ ]:
Rstore.head()

In [ ]:
Rstore.shape

In [ ]:
Rstore.to_csv('proj_result.csv')

In [ ]:
adata2.obs['Cell_type'] = Rstore['CT'].values

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sc.pl.umap(adata2, color="Cell_type", legend_loc="on data", legend_fontsize="xx-small", ax=ax, show=False)

In [ ]:
adata1.write('./write/GSE159032.h5ad')
adata2.write('./write/other.h5ad')

In [ ]:
adata1 = sc.read('./write/GSE159032.h5ad')
adata2 = sc.read('./write/other.h5ad')

In [ ]:
adata = anndata.concat([adata1,adata2])

In [ ]:
adata.write('./write/All_outer_leidenv2.h5ad')

# Extract macrophages and filter

In [ ]:
adata = sc.read('./write/All_outer_leidenv2.h5ad')

In [ ]:
adata1 = adata[adata.obs.Cell_type=='Macrophages']

In [ ]:
adata1.shape

In [ ]:
adata1 = adata1[adata1.obs.Genotype == 'WT']

In [ ]:
adata1 = adata1[~np.in1d(adata1.obs.Treatment,['DMSO','IWR','morphine'])]

In [ ]:
adata1 = adata1[(adata1.obs.Tissue == 'heart')|(adata1.obs.Author != 'BoHu')]

In [ ]:
adata1.write('./Final_write/ZF_mac.h5ad')